In [1]:
require 'nn'
require 'rnn'

In [2]:
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

In [3]:
trueqValues = torch.rand(10, 2)              ---- Simulating the data
qMaxtrue, qindxtrue = torch.max(trueqValues, 2)  --- Generating the max values and getting the indices
--- I want to select the qindx elements for each row
trueactions = torch.zeros(10, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))

-- Simulating streams and queries
sentences = genNbyK(10, 15, 1, 100)
queries = genNbyK(10, 4, 1, 100)

In [4]:
function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

In [5]:
model = buildModel('bow', 100, 50, 'f1', false, false)

Running bag-of-words model to learn f1	


In [9]:
preds = model:forward({sentences, queries, torch.zeros(10, 5)})
print(preds)

 0.0171 -0.0070
 0.0097 -0.1079
-0.0103 -0.2170
 0.0678  0.4116
 0.1033 -0.1495
-0.0557 -0.1621
-0.1048 -0.0276
-0.0548 -0.1336
 0.0142  0.0134
 0.3362  0.1590
[torch.DoubleTensor of size 10x2]



In [7]:
qMax, qindx = torch.max(preds, 2)
actions = torch.zeros(10, 2):scatter(2, qindx, torch.ones(preds:size()))

In [16]:
sentences

  44   16   60   62   93  100   78   75   48   48   23   42   24   43   70
  44   95   31   80   84    8   83   16   67   12   79   91   42   89   23
  20   50   15  100   63    4   89   71   86   52   57   75   66   77   12
  62   48   36   21   83   42    6   16   42   43   71   17   27   25   47
  78   55   14   83    2   30   97   70   38   72   83   25    3   72   98
  48   91    4   23   78   66   31   74   71   47   66   20   90   69   42
  98   40   84    1   99   95   11   91   78   54   16   20   86  100   30
  67   80   57   13   19  100   59   87   15   99   70   65   12   37   78
  37   35   53   24   57   15   81   34   54   21   30   50  100   89   23
  36   92   64   92   38   20   43   29   51   40   45    4   88   81   82
[torch.LongTensor of size 10x15]



In [14]:
indx = torch.range(1, 10)
-- Summary index for selecting the words
sumindx = nn.MaskedSelect():forward({indx, actions:select(2,1):resize(10, 1):byte()})  --- sentences
-- need a LongTensor to do the selection
sumindx = torch.LongTensor(sumindx:size(1)):copy(sumindx)
--- This gives the wrong selection we want
print(sentences:index(1, sumindx))

  44   16   60   62   93  100   78   75   48   48   23   42   24   43   70
  20   50   15  100   63    4   89   71   86   52   57   75   66   77   12
  78   55   14   83    2   30   97   70   38   72   83   25    3   72   98
  67   80   57   13   19  100   59   87   15   99   70   65   12   37   78
  37   35   53   24   57   15   81   34   54   21   30   50  100   89   23
[torch.LongTensor of size 5x15]

